In [57]:
import requests
import json
import time
from bs4 import BeautifulSoup

In [58]:
base_url = "https://kurser.dtu.dk/course/"
cookies_base = {
    'ASP.NET_SessionId' : "re4u1kgeqp3kwme1ryk3a3lc",
    '{DTUCoursesPublicLanguage}' : 'en-GB'
}
cookies_eval = {
    'ASP.NET_SessionId' : "xvryd0ayizd0hz53me0uboz5",
    '{DTUCoursesPublicLanguage}' : 'en-GB',
}
cookies_grades = {
    'ASP.NET_SessionId' : "c32erhch2dq3slvg5wjkkp1c",
    '{DTUCoursesPublicLanguage}' : 'en-GB'
}

In [59]:
with open('valid_courses.json') as f:
    valid_courses = json.load(f)

In [80]:
eval_links = {}

for course_num in valid_courses:
    
    # Define page content
    course_url = f"{base_url}{course_num}/info"
    response = requests.get(course_url, cookies=cookies_base)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find left div, and all links within it
    left_box = soup.find('div', class_='box')
    atags = left_box.find_all('a')
        
    # Extract link-urls and link-texts
    eval_hrefs = []
    eval_texts = []
    grade_hrefs = []
    grade_texts = []
    for atag in atags:
        if "evaluering.dtu.dk" in atag['href']:
            eval_hrefs.append(atag['href'])
            eval_texts.append(atag.text.replace('\r', '').replace('\n', '').replace('  ', ''))
        elif "karakterer.dtu.dk" in atag['href']:
            grade_hrefs.append(atag['href'])
            grade_texts.append(atag.text.replace('\r', '').replace('\n', '').replace('  ', ''))

    # print(eval_hrefs)
    # print(eval_texts)
    # print(grade_hrefs)
    # print(grade_texts)
    
    # Get content of the links
    print(course_url)
    eval_response = requests.get(eval_hrefs[0], cookies=cookies)
    grade_response1 = requests.get(grade_hrefs[0], cookies=cookies_grades)
    if len(grade_hrefs) > 1:
        grade_response2 = requests.get(grade_hrefs[1], cookies=cookies_grades)
        
        # Second latest grade
        grade2_soup = BeautifulSoup(grade_response2.content, 'html.parser')
        form_2 = grade2_soup.find('form', id='karsumForm')
        num_participants_2 = int(form_2.find_all('td')[1].text.replace('\r', '').replace('\n', '').replace('  ', ''))
    else:
        num_participants_2 = 0
    
    # Latest grade
    grade1_soup = BeautifulSoup(grade_response1.content, 'html.parser')
    form_1 = grade1_soup.find('form', id='karsumForm')
    num_participants_1 = int(form_1.find_all('td')[1].text.replace('\r', '').replace('\n', '').replace('  ', ''))
    
    
    
    # Based on the number of participants, choose the correct grade page
    if num_participants_1*2 > num_participants_2:
        eval_links[course_num] = {'eval': eval_response.text.replace('\r', '').replace('\n', '').replace('  ', ''), 'grade': grade_response1.text.replace('\r', '').replace('\n', '').replace('  ', '')}
    else:
        eval_links[course_num] = {'eval': eval_response.text.replace('\r', '').replace('\n', '').replace('  ', ''), 'grade': grade_response2.text.replace('\r', '').replace('\n', '').replace('  ', '')}   
    

https://kurser.dtu.dk/course/01001/info
https://kurser.dtu.dk/course/01002/info
https://kurser.dtu.dk/course/01003/info
https://kurser.dtu.dk/course/01004/info
https://kurser.dtu.dk/course/01017/info
https://kurser.dtu.dk/course/01018/info
https://kurser.dtu.dk/course/01019/info
https://kurser.dtu.dk/course/01020/info
https://kurser.dtu.dk/course/01025/info
https://kurser.dtu.dk/course/01034/info
https://kurser.dtu.dk/course/01035/info
https://kurser.dtu.dk/course/01037/info
https://kurser.dtu.dk/course/01125/info
https://kurser.dtu.dk/course/01227/info
https://kurser.dtu.dk/course/01237/info
https://kurser.dtu.dk/course/01238/info
https://kurser.dtu.dk/course/01257/info
https://kurser.dtu.dk/course/01325/info
https://kurser.dtu.dk/course/01405/info
https://kurser.dtu.dk/course/01410/info
https://kurser.dtu.dk/course/01415/info
https://kurser.dtu.dk/course/01418/info
https://kurser.dtu.dk/course/01426/info
https://kurser.dtu.dk/course/01527/info
https://kurser.dtu.dk/course/01617/info


IndexError: list index out of range

In [76]:
eval_links

{'01001': {'eval': '<!DOCTYPE html><html id="html" xmlns="http://www.w3.org/1999/xhtml" lang="da" xml:lang="da"><head>\t<meta http-equiv="X-UA-Compatible" content="IE=edge" />\t<!--googleoff: index-->\t<title>\t\tKursusevalueringer - Studieinformation</title>\t<meta charset="UTF-8" />\t<meta name="viewport" content="width=device-width, initial-scale=1, minimum-scale=1, maximum-scale=1" />\t<meta name="og:description" content="" /><meta name="keywords" content="" /><meta name="DTUKeywords" content="" /><meta name="dtucontenttype" content="ArticleHtml" /><meta name="pid" content="df2e7462-00cb-403a-aa3e-a7072356f3e1"/><meta name="lid" content=""/><meta property="og:title" content="Kursusevalueringer - Studieinformation" /><meta property="og:type" content="ArticleHtml" /><meta property="og:url" content="https://sites.dtu.dk/sdb-scrape-dk/service/NoegletalEkstern?hidecontent=true" /><meta property="og:image" content="" /><meta property="og:site_name" content="https://sites.dtu.dk" /><meta 

In [61]:
# save dict as json
with open('eval_links.json', 'w') as file:
    json.dump(eval_links, file)